In [ ]:
from dotenv import load_dotenv
import os


In [10]:
model_name = os.getenv("gpt_deployment_name")
openai_api_key = os.environ["OPENAI_API_KEY"]

In [11]:
print("API Key loaded:", openai_api_key[:8] + "...")
print(f"Model name: {model_name}")

API Key loaded: sk-proj-...
Model name: gpt-3.5-turbo


In [ ]:

import openai
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "system", "content": "You are an assistant chatbot"},
    {"role": "user", "content": "hello"}
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I help you today?


In [9]:
#load the comments_table.pickle
import pickle
from pathlib import Path
import pandas as pd

table_name = "full_comments_renamed.pickle"
data_dir = Path('../Data')
file_path = data_dir / table_name

with open(file=file_path, mode="rb") as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df.head()

,ProductID,FullComment
0,000010001,Warning: Unindexed pattern. Unit Cell Data Sou...
1,000010002,Warning: Unindexed pattern. Unit Cell Data Sou...
2,000010003,Warning: Unindexed pattern. Unit Cell Data Sou...
3,000010004,Warning: Unindexed pattern. Unit Cell Data Sou...
4,000010005,Warning: Unindexed pattern. Unit Cell Data Sou...


In [10]:
import json
row = df.iloc[-1].to_dict()
print(json.dumps(row, indent=2, ensure_ascii=False))

{
  "ProductID": "050104886",
  "FullComment": "Color: colourless. General Comments: Reported GOF=0.954. Habit: Prismatic. Structures: 10.1107/S2056989019007989. Article Title: Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate. Temperature of Data Collection: Pattern taken at 296 K. Unit Cell Data Source: Single Crystal."
}


In [132]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

full_comment = """Color: colourless. General Comments: Reported GOF=0.954. Habit: Prismatic. Structures: 10.1107/S2056989019007989. Article Title: Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate. Temperature of Data Collection: Pattern taken at 296 K. Unit Cell Data Source: Single Crystal."""

messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant that extracts structured metadata from materials science comment fields. "
            "Parse the input text and return a JSON object with appropriate keys based on recognized categories such as: "
            "Color, Habit, General Comments, Structures, Temperature of Data Collection, Unit Cell Data Source, Article Title, etc. "
            "If a field isn't present, omit it from the output."
        )
    },
    {
        "role": "user",
        "content": full_comment
    }
]

response_gpt35 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

print(response_gpt35.choices[0].message.content)

{
  "Color": "colourless",
  "General Comments": "Reported GOF=0.954",
  "Habit": "Prismatic",
  "Structures": "10.1107/S2056989019007989",
  "Article Title": "Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate",
  "Temperature of Data Collection": "296 K",
  "Unit Cell Data Source": "Single Crystal"
}


In [133]:
df_test = df[-10:]
df_test.shape

(10, 2)

# Full Code to Enrich Each Row and Extend It

In [134]:
from openai import OpenAI
import os
import json
import pandas as pd
from tqdm import tqdm

# Set up OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Convert DataFrame rows to dicts
rows = df_test.to_dict(orient="records")

# Store enriched rows
enriched_rows_gpt = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_gpt.append(row)
        continue

    messages = [
        {
            "role": "system",
            "content": (
            "You are a helpful assistant that extracts structured metadata from materials science comment fields. "
            "Parse the input text and return a JSON object with appropriate keys based on recognized categories such as: "
            "Color, Habit, General Comments, Structures, Temperature of Data Collection, Unit Cell Data Source, Article Title, etc. "
            "If a field isn't present, omit it from the output."
        )
        },
        {"role": "user", "content": full_comment}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Parse the response into a dictionary
        parsed_data = json.loads(response.choices[0].message.content)

        # Extend the original row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows_gpt.append(row)

# Convert back to DataFrame
enriched_df_gpt = pd.DataFrame(enriched_rows_gpt)


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:16<00:00,  1.68s/it]


In [136]:
%%time
# Store enriched rows
enriched_rows_gpt = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_gpt.append(row)
        continue

    messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant that extracts structured metadata from materials science comment fields. "
            "Parse the input text and return a JSON object with appropriate keys based on recognized categories such as: "
            "Color, Habit, General Comments, Structures, Temperature of Data Collection, Unit Cell Data Source, Article Title, etc. "
            "If a field isn't present, omit it from the output."
        )
    },
    {
        "role": "user",
        "content": full_comment
    }
]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Parse the response into a dictionary
        parsed_data = json.loads(response.choices[0].message.content)

        # Extend the original row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows_gpt.append(row)

# Convert back to DataFrame
enriched_df_gpt = pd.DataFrame(enriched_rows_gpt)

100%|██████████| 10/10 [00:15<00:00,  1.53s/it]

CPU times: user 86.2 ms, sys: 15.3 ms, total: 101 ms
Wall time: 15.3 s


In [26]:
enriched_df.iloc[-1]

ProductID                                                                 050104886
FullComment                       Color: colourless. General Comments: Reported ...
Color                                                                    colourless
General Comments                                                 Reported GOF=0.954
Habit                                                                     Prismatic
Structures                                                10.1107/S2056989019007989
Article Title                     Crystal structure and DFT study of benzyl 1-be...
Temperature of Data Collection                                                296 K
Unit Cell Data Source                                                Single Crystal
Minor Warning                                                                   NaN
Name: 9, dtype: object

## estimate the cost

In [27]:
import tiktoken

def count_tokens(messages, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens_per_message = 4
    tokens_per_name = -1
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 2  # for the assistant priming
    return num_tokens


In [75]:
total_input_tokens = 0
total_output_tokens = 0
# Store enriched rows
enriched_rows = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows.append(row)
        continue

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that extracts structured metadata from materials science comment fields. "
                "Parse the input text and return a JSON object with appropriate keys based on recognized categories such as: "
                "Color, Habit, General Comments, Structures, Temperature of Data Collection, Unit Cell Data Source, Article Title, etc. "
                "If a field isn't present, omit it from the output."
            )
        },
        {"role": "user", "content": full_comment}
    ]

    try:
        # Count input tokens
        input_tokens = count_tokens(messages, model="gpt-3.5-turbo")

        # Call API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        output_text = response.choices[0].message.content
        output_tokens = len(tiktoken.encoding_for_model("gpt-3.5-turbo").encode(output_text))

        total_input_tokens += input_tokens
        total_output_tokens += output_tokens

        # Parse and update row
        parsed_data = json.loads(output_text)
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows.append(row)


100%|██████████| 10/10 [00:22<00:00,  2.21s/it]


In [29]:
# OpenAI pricing (April 2024)
input_cost = total_input_tokens / 1000 * 0.0005
output_cost = total_output_tokens / 1000 * 0.0015
total_cost = input_cost + output_cost

print(f"Total input tokens: {total_input_tokens}")
print(f"Total output tokens: {total_output_tokens}")
print(f"Estimated cost: ${total_cost:.6f}")


Total input tokens: 1934
Total output tokens: 1328
Estimated cost: $0.002959


In [32]:
df.shape[0]/10 * total_cost

326.7141383

# Try with Ollama (free models)

In [48]:
from ollama import Client

client = Client()

full_comment = """Color: colourless. General Comments: Reported GOF=0.954. Habit: Prismatic. Structures: 10.1107/S2056989019007989. Article Title: Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate. Temperature of Data Collection: Pattern taken at 296 K. Unit Cell Data Source: Single Crystal."""

prompt = f"""
You are a scientific assistant trained to extract structured metadata from materials science comments.

Your task is to read the following input comment and extract any clearly stated metadata values. 
Return a JSON object that contains only the fields found in the comment.

The metadata may relate to:
- **Physical and chemical properties** (e.g. Color, Melting Point, Boiling Point, Sublimation Point, Refractive Index, Reflectance, Mohs Hardness, Vickers Hardness Number)
- **Crystallography and structure** (e.g. Unit Cell, Unit Cell Data Source, Structures, Wyckoff Sequence, ANX, Atomic Position)
- **Processing and sample information** (e.g. Sample Preparation, Sample Source or Locality, Sensitivity, Pressure of Datacollection)
- **Analysis metadata** (e.g. General Comments, Scale Factors, Calculated Pattern Original Remarks, Article Title, Test from ICSD, LPF Collection Code, ICSD Collection Code)
- **Warnings or classifications** (e.g. Minor Warning, Significant Warning, Major Warning, Reason O Quality Was Assigned, Deleted or Rejected By)
- **Additional characteristics** (e.g. Habit, Bioactivity, Common Name, Note, Polymorphism/Phase Transition, Thermal Patterns)

Only extract data that is **explicitly stated**. Do not infer or guess.

Return only the fields that are mentioned in the comment in a clean JSON dictionary.

Example format:
{{
  "Color": "pale yellow",
  "Habit": "Needle-like",
  "General Comments": "Pattern taken at 298K using Cu Kα radiation",
  "Structures": "10.1107/S2056989019007989",
  "Melting Point": "120 °C",
  "Sample Source or Locality": "Istanbul, Turkey"
}}

Here is the comment:
{full_comment}
"""


response_mistral = client.chat(model='mistral', messages=[
    {"role": "user", "content": prompt}
])

print(response_mistral['message']['content'])

 {
      "Color": "colourless",
      "Habit": "Prismatic",
      "General Comments": "Reported GOF=0.954",
      "Structures": "10.1107/S2056989019007989",
      "Article Title": "Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate",
      "Temperature of Data Collection": "296 K"
      "Unit Cell Data Source": "Single Crystal"
   }


In [49]:
print(response_gpt35.choices[0].message.content)

{
  "Color": "colourless",
  "General Comments": "Reported GOF=0.954",
  "Habit": "Prismatic",
  "Structures": "10.1107/S2056989019007989",
  "Article Title": "Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate",
  "Temperature of Data Collection": "Pattern taken at 296 K",
  "Unit Cell Data Source": "Single Crystal"
}


In [50]:
response_mistral['message']['content'] == response_gpt35.choices[0].message.content


False

In [76]:
from ollama import Client
import json
import pandas as pd
from tqdm import tqdm

# Initialize Ollama client
client = Client()

# Convert DataFrame rows to dicts
rows = df_test.to_dict(orient="records")
enriched_rows_mistral = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_mistral.append(row)
        continue

    # Define the full structured prompt
    prompt = f"""
You are a scientific assistant trained to extract structured metadata from materials science comments.

Your task is to read the following input comment and extract any clearly stated metadata values. 
Return a JSON object that contains only the fields found in the comment.

The metadata may relate to:
- **Physical and chemical properties** (e.g. Color, Melting Point, Boiling Point, Sublimation Point, Refractive Index, Reflectance, Mohs Hardness, Vickers Hardness Number)
- **Crystallography and structure** (e.g. Unit Cell, Unit Cell Data Source, Structures, Wyckoff Sequence, ANX, Atomic Position)
- **Processing and sample information** (e.g. Sample Preparation, Sample Source or Locality, Sensitivity, Pressure of Datacollection)
- **Analysis metadata** (e.g. General Comments, Scale Factors, Calculated Pattern Original Remarks, Article Title, Test from ICSD, LPF Collection Code, ICSD Collection Code)
- **Warnings or classifications** (e.g. Minor Warning, Significant Warning, Major Warning, Reason O Quality Was Assigned, Deleted or Rejected By)
- **Additional characteristics** (e.g. Habit, Bioactivity, Common Name, Note, Polymorphism/Phase Transition, Thermal Patterns)

Only extract data that is **explicitly stated**. Do not infer or guess.

Return a valid JSON object using double quotes for all keys and values. Do not include trailing commas. Do not wrap the JSON in markdown or code blocks.

Example format:
{{
  "Color": "pale yellow",
  "Habit": "Needle-like",
  "General Comments": "Pattern taken at 298K using Cu Kα radiation",
  "Structures": "10.1107/S2056989019007989",
  "Melting Point": "120 °C",
  "Sample Source or Locality": "Istanbul, Turkey"
}}

Here is the comment:
{full_comment}
"""

    try:
        # Query Ollama with the prompt
        response = client.chat(
            model='mistral',
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse the response as JSON
        parsed_data = json.loads(response['message']['content'])

        # Update the row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows.append(row)

# Convert back to DataFrame
enriched_df = pd.DataFrame(enriched_rows_mistral)


 50%|█████     | 5/10 [00:16<00:14,  3.00s/it]

Error processing ProductID 050104881: Expecting ',' delimiter: line 8 column 7 (char 357)


 70%|███████   | 7/10 [00:22<00:08,  2.96s/it]

Error processing ProductID 050104883: Expecting ',' delimiter: line 8 column 6 (char 311)


 90%|█████████ | 9/10 [00:28<00:02,  2.98s/it]

Error processing ProductID 050104885: Expecting ',' delimiter: line 8 column 7 (char 380)


100%|██████████| 10/10 [00:31<00:00,  3.13s/it]


In [54]:
df_test.iloc[2].to_dict()

{'ProductID': '050104879',
 'FullComment': 'Color: colourless. General Comments: Reported GOF=1.110. Habit: Block. Structures: 10.1107/S205698901900762X. Article Title: Crystal structure of N -(diphenylphosphoryl)-2-methoxybenzamide. Temperature of Data Collection: Pattern taken at 294 K. Unit Cell Data Source: Single Crystal.'}

## one-shot learning

In [125]:
%%timeit
from ollama import Client
import json
import pandas as pd
from tqdm import tqdm

# Initialize Ollama client
client = Client()

# Convert DataFrame rows to dicts
rows = df_test.to_dict(orient="records")
enriched_rows_mistral = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_mistral.append(row)
        continue

    # Define the full structured prompt
    prompt = fprompt = f"""
You are a scientific assistant that extracts structured metadata from materials science comments.

Your task is to read each input comment and extract only the explicitly stated metadata. Return the result as a clean JSON object with only the mentioned fields.

The metadata may include fields like:
- Color, Habit, General Comments, Structures, Article Title, Temperature of Data Collection, Unit Cell Data Source
- (Also: Melting Point, Refractive Index, Sample Preparation, Warning, etc.)

Only extract data that is **explicitly stated**. Do not infer or guess.

Return only the fields that are mentioned in the comment in a clean JSON dictionary.

Here is an example:

Input:
Color: colourless. General Comments: Reported GOF=0.954. Habit: Prismatic. Structures: 10.1107/S2056989019007989. Article Title: Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate. Temperature of Data Collection: Pattern taken at 296 K. Unit Cell Data Source: Single Crystal.

Output:
{{
  "Color": "colourless",
  "General Comments": "Reported GOF=0.954",
  "Habit": "Prismatic",
  "Structures": "10.1107/S2056989019007989",
  "Article Title": "Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate",
  "Temperature of Data Collection": "Pattern taken at 296 K",
  "Unit Cell Data Source": "Single Crystal"
}}

Now do the same for the following comment:

Input:
{full_comment}

Output:
"""

    try:
        # Query Ollama with the prompt
        response = client.chat(
            model='mistral',
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse the response as JSON
        parsed_data = json.loads(response['message']['content'])

        # Update the row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows_mistral.append(row)

# Convert back to DataFrame
enriched_df_mistral = pd.DataFrame(enriched_rows_mistral)


 80%|████████  | 8/10 [00:27<00:06,  3.45s/it]


KeyboardInterrupt: 

In [106]:
enriched_df_mistral

,ProductID,FullComment,Color,General Comments,Habit,Structures,Article Title,Temperature of Data Collection,Unit Cell Data Source,Minor Warning
0,050104877,Color: colorless. General Comments: Reported G...,colorless,Reported GOF=1.069,Block,10.1107/S2056989019006194,Crystal structure of bis(4-methoxypyridine-kap...,Pattern taken at 170 K,Single Crystal,NaN
1,050104878,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=0.940,Plate,10.1107/S205689019004432,Crystal structures of the synthetic intermedia...,Pattern taken at 110 K,Single Crystal,NaN
2,050104879,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.110,Block,10.1107/S205698901900762X,Crystal structure of N -(diphenylphosphoryl)-2...,Pattern taken at 294 K,Single Crystal,NaN
3,050104880,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.047,Block,10.1107/S2056989019007783,Crystal structure and Hirshfeld surface analys...,Pattern taken at 294 K,Single Crystal,NaN
4,050104881,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.042,Block,10.1107/S2056989019007758,Structure and Hirshfeld surface analysis of th...,Pattern taken at 100 K,Single Crystal,NaN
5,050104882,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.029,Prismatic,10.1107/S205689019007801,"Crystal structure, Hirshfeld surface analysis ...",Pattern taken at 296.15 K,Single Crystal,Reported displacement parameters on non H atom...
6,050104883,Color: yellow. General Comments: Reported GOF=...,yellow,Reported GOF=1.047,Block,10.1107/S2056989019007503,Some chalcones derived from thiophene-3-carbal...,Pattern taken at 294.0 K,Single Crystal,NaN
7,050104884,Color: brown. General Comments: Reported GOF=1...,brown,Reported GOF=1.035,Prismatic,10.1107/S2056989019007692,"Synthesis, crystal structure and Hirshfeld sur...",Pattern taken at 296 K,Single Crystal,NaN
8,050104885,Color: dark blue. General Comments: Reported G...,dark blue,Reported GOF=1.036,Plate,10.1107/S2056989019008053,"Synthesis, characterization, and crystal struc...",Pattern taken at 150.00 K,Single Crystal,NaN
9,050104886,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=0.954,Prismatic,10.1107/S2056989019007989,Crystal structure and DFT study of benzyl 1-be...,Pattern taken at 296 K,Single Crystal,NaN


In [82]:
enriched_df_gpt

,ProductID,FullComment,Color,General Comments,Habit,Structures,Article Title,Temperature of Data Collection,Unit Cell Data Source
0,050104877,Color: colorless. General Comments: Reported G...,colorless,Reported GOF=1.069,Block,10.1107/S2056989019006194,Crystal structure of bis(4-methoxypyridine-kap...,Pattern taken at 170 K,Single Crystal
1,050104878,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=0.940,Plate,10,Crystal structures of the synthetic intermedia...,Pattern taken at 110 K,Single Crystal
2,050104879,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.110,Block,10.1107/S205698901900762X,Crystal structure of N -(diphenylphosphoryl)-2...,294 K,Single Crystal
3,050104880,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.047,Block,10.1107/S2056989019007783,Crystal structure and Hirshfeld surface analys...,Pattern taken at 294 K,Single Crystal
4,050104881,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.042,Block,10.1107/S2056989019007758,Structure and Hirshfeld surface analysis of th...,100 K,Single Crystal
5,050104882,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=1.029,Prismatic,10.1107/S2056989019007801,"Crystal structure, Hirshfeld surface analysis ...",296.15 K,Single Crystal
6,050104883,Color: yellow. General Comments: Reported GOF=...,yellow,{'Reported GOF': 1.047},Block,10,Some chalcones derived from thiophene-3-carbal...,294.0 K,Single Crystal
7,050104884,Color: brown. General Comments: Reported GOF=1...,brown,Reported GOF=1.035,Prismatic,10.1107/S2056989019007692,"Synthesis, crystal structure and Hirshfeld sur...",Pattern taken at 296 K,Single Crystal
8,050104885,Color: dark blue. General Comments: Reported G...,dark blue,Reported GOF=1.036,Plate,10.1107/S2056989019008053,"Synthesis, characterization, and crystal struc...",Pattern taken at 150.00 K,Single Crystal
9,050104886,Color: colourless. General Comments: Reported ...,colourless,Reported GOF=0.954,Prismatic,10.1107/S2056989019007989,Crystal structure and DFT study of benzyl 1-be...,296 K,Single Crystal


In [131]:
columns_to_compare = [
    "ProductID",
    "FullComment",
    "Color",
    "General Comments",
    "Habit",
    "Structures",
    "Article Title",
    "Temperature of Data Collection",
    "Unit Cell Data Source"
]

row1 = enriched_df_gpt.loc[0, columns_to_compare].to_dict()
row2 = enriched_df_mistral.loc[0, columns_to_compare].to_dict()

def compare_selected_columns(r1, r2):
    differences = {}
    for key in r1:
        if r1[key] != r2[key]:
            differences[key] = (r1[key], r2[key])
    return differences

diffs = compare_selected_columns(row1, row2)

if not diffs:
    print("All selected fields match in the first row.")
else:
    print("Differences found in the first row:")
    for key, (v1, v2) in diffs.items():
        print(f"  {key}: df1 -> {v1} | df2 -> {v2}")


Differences found in the first row:
  Color: df1 -> colorless | df2 -> nan
  General Comments: df1 -> Reported GOF=1.069 | df2 -> nan
  Habit: df1 -> Block | df2 -> nan
  Structures: df1 -> 10.1107/S2056989019006194 | df2 -> nan
  Article Title: df1 -> Crystal structure of bis(4-methoxypyridine-kappa N)(meso -5,10,15,20-tetraphenylporphyrinato-kappa4N,N`,N``,N```)iron(III) perchlorate | df2 -> nan
  Temperature of Data Collection: df1 -> Pattern taken at 170 K | df2 -> nan
  Unit Cell Data Source: df1 -> Single Crystal | df2 -> nan


In [109]:
for i in range(len(enriched_df_gpt)):
    row_gpt = enriched_df_gpt.loc[0, columns_to_compare].to_dict()
    row_mistral = enriched_df_mistral.loc[0, columns_to_compare].to_dict()
    diffs = compare_selected_columns(row_gpt, row_mistral)

    if not diffs:
        print(f"All selected fields match in the row number: {i}.")

All selected fields match in the row number: 0.
All selected fields match in the row number: 1.
All selected fields match in the row number: 2.
All selected fields match in the row number: 3.
All selected fields match in the row number: 4.
All selected fields match in the row number: 5.
All selected fields match in the row number: 6.
All selected fields match in the row number: 7.
All selected fields match in the row number: 8.
All selected fields match in the row number: 9.


In [114]:
estimated_time = len(df)*30.8/10
print(f"Estimated time: {estimated_time:.1f} sec or {estimated_time/3600:.1f} hours")

Estimated time: 3400742.0 sec or 944.7 hours


# Testing ligher and faster models

## testing phi



In [119]:
%%time
from ollama import Client
import json
import pandas as pd
from tqdm import tqdm

# Initialize Ollama client
client = Client()

# Convert DataFrame rows to dicts
rows = df_test.to_dict(orient="records")
enriched_rows_mistral = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_mistral.append(row)
        continue

    # Define the full structured prompt
    prompt = f"""
Extract structured metadata from the following scientific comment.

Return only a JSON object with the fields that are explicitly mentioned in the comment. Use double quotes. Do not include explanations.

Each key is followed by a ":" and each value followed by a "."
"""

    try:
        # Query Ollama with the prompt
        response = client.chat(
            model='phi',
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse the response as JSON
        parsed_data = json.loads(response['message']['content'])

        # Update the row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows_mistral.append(row)

# Convert back to DataFrame
enriched_df_mistral = pd.DataFrame(enriched_rows_mistral)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:08<01:13,  8.14s/it]

Error processing ProductID 050104877: Expecting value: line 1 column 2 (char 1)


 40%|████      | 4/10 [00:09<00:08,  1.35s/it]

Error processing ProductID 050104880: Extra data: line 1 column 4 (char 3)


...

Error processing ProductID 050104883: Extra data: line 1 column 12 (char 11)


 80%|████████  | 8/10 [00:11<00:01,  1.60it/s]

Error processing ProductID 050104884: Extra data: line 1 column 57 (char 56)


100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

CPU times: user 40.7 ms, sys: 8.12 ms, total: 48.8 ms
Wall time: 11.6 s


In [130]:
%%time
from ollama import Client
import json
import pandas as pd
from tqdm import tqdm

# Initialize Ollama client
client = Client()

# Convert DataFrame rows to dicts
rows = df_test.to_dict(orient="records")
enriched_rows_mistral = []

for row in tqdm(rows):
    product_id = row.get("ProductID", "")
    full_comment = row.get("FullComment", "")

    if not full_comment:
        enriched_rows_mistral.append(row)
        continue

    # Define the full structured prompt
    prompt = f"""Extract fields as JSON.

    Keys are: General Comments, Additional Patterns, ANX, Analysis,
30
Atomic Position, Bioactivity, Boiling Point, Formula from original source, Color, Common Name, Deleted Or Rejected
By, Delete duplicate, Scattering Factors, Footnotes for D-spacings and Intensities, ICSD Collection Code, Habit,
Footnotes for HKL and Superlattice D- spacings, Hyperthetical Structure, LPF Collection Code, Water Loss, Additional
Diffraction Lines, NIST M&A collection code, Mohs hardness, Melting Point, Note, Other Cell, Opaque Optical Data, Reason O Quality
Was Assigned, Powder Data, Polymorphism/Phase Transition, Physical Property, Sample Preparation, Pressure of Datacollection,
Refractive Index, Reflectance, Calculated Pattern Original Remarks, Scale Factors, Sample Source or Locality, Sensitivity, Sublimation
Point, Structures, Temperature Factor, Thermal Patterns, Temperature of Data Collection, Test from ICSD, Unit Cell, Vickers Hardness Number, 
Minor Warning, Significant Warning, Major Warning, Warning, Wyckoff Sequence, Raw Data Comment

Text:
{full_comment}

JSON:
"""

    try:
        # Query Ollama with the prompt
        response = client.chat(
            model='phi',
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse the response as JSON
        parsed_data = json.loads(response['message']['content'])

        # Update the row with parsed fields
        row.update(parsed_data)

    except Exception as e:
        print(f"Error processing ProductID {product_id}: {e}")

    enriched_rows_mistral.append(row)

# Convert back to DataFrame
enriched_df_mistral = pd.DataFrame(enriched_rows_mistral)


 10%|█         | 1/10 [00:06<01:01,  6.87s/it]

Error processing ProductID 050104877: Expecting property name enclosed in double quotes: line 11 column 8 (char 469)


 20%|██        | 2/10 [00:15<01:01,  7.63s/it]

Error processing ProductID 050104878: Expecting property name enclosed in double quotes: line 56 column 1 (char 1628)


 30%|███       | 3/10 [00:35<01:35, 13.62s/it]

...

Error processing ProductID 050104881: Expecting value: line 1 column 2 (char 1)


 60%|██████    | 6/10 [01:07<00:50, 12.56s/it]

Error processing ProductID 050104882: Expecting ',' delimiter: line 82 column 5 (char 2425)


 60%|██████    | 6/10 [04:13<02:49, 42.31s/it]


KeyboardInterrupt: 

In [127]:
df.iloc[-1].to_dict()

{'ProductID': '050104886',
 'FullComment': 'Color: colourless. General Comments: Reported GOF=0.954. Habit: Prismatic. Structures: 10.1107/S2056989019007989. Article Title: Crystal structure and DFT study of benzyl 1-benzyl-2-oxo-1,2-dihydroquinoline-4-carboxylate. Temperature of Data Collection: Pattern taken at 296 K. Unit Cell Data Source: Single Crystal.'}